<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/ml_challenge_ResNet_50_with_3_channels_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

/Applications/anaconda3/lib/python3.9/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


KeyError: ignored

In [2]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
!pip install tqdm
import rasterio
from rasterio.plot import reshape_as_image
from tqdm import tqdm

!pip install torchgeo
import torchgeo.models

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor

  Using cached rasterio-1.2.10-cp39-cp39-macosx_10_9_x86_64.whl (21.2 MB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached affine-2.3.1-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 204 kB 2.7 MB/s 
     |████████████████████████████████| 74 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 20.5 MB/s 
     |████████████████████████████████| 431 kB 77.6 MB/s 
     |████████████████████████████████| 409 kB 58.5 MB/s 
     |████████████████████████████████| 88 kB 20.0 MB/s 
     |████████████████████████████████| 584 kB 65.6 MB/s 
     |████████████████████████████████| 8.2 MB 815 kB/s 
     |████████████████████████████████| 18.5 MB 25.5 MB/s 
     |████████████████████████████████| 493 kB 51.5 MB/s 
     |████████████████████████████████| 112 kB 14.0 MB/s 
     |████████████████████████████████| 58 kB 14.2 MB/s 
     |

In [3]:
!pip install wandb
!wandb login
import wandb
#Key: 9da448bfaa162b572403e1551114a17058f249d0

wandb: Currently logged in as: nicikess. Use `wandb login --relogin` to force relogin


In [4]:
#local
#Loads tiff folders

import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
#folder = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/AnnualCrop
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/Forest
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/HerbaceousVegetation
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/Highway
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/Industrial
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/Pasture
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/PermanentCrop
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/Residential
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/River
/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset/SeaLake


In [3]:
#Define paths

#ROOT_PATH = '/content/drive/MyDrive/ML/'
ROOT_PATH = '/Volumes/GoogleDrive/My Drive/ML'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

#file_path_train = '/content/drive/MyDrive/ML/data/dataset/train.csv'
#file_path_test = '/content/drive/MyDrive/ML/data/dataset/test.csv'
file_path_train = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/train.csv'
file_path_test = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/test.csv'
data_df_train = pd.read_csv(file_path_train)
data_df_test = pd.read_csv(file_path_test)

data_df_train.columns = data_df_train.columns.str.replace('path', 'image_id')
data_df_train.columns = data_df_train.columns.str.replace('Label', 'label')

In [6]:
data_df_train.head()

,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [7]:
type(data_df_train)

pandas.core.frame.DataFrame

In [7]:
#Init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [27]:
#Classes and labels

IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [25]:
#Encoder and decoder

## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

In [ ]:
#Open tiff files with rasterio and safe to dictionary

'''import os

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    with rasterio.open(pic, "r") as img:
      arr[i] = np.moveaxis(img.read(), 0, 2)
      i += 1
      
  data[index] = arr'''

'import os\n\ndata = {}\n\nfor i in range(len(subfolders)):\n\n  txtfiles = []\n\n  # traverse whole directory\n  for root, dirs, files in os.walk(subfolders[i]):\n    # select file name\n    for file in files:\n        # check the extension of files\n        if file.endswith(\'.tif\'):\n            txtfiles.append(os.path.join(root, file))\n\n  code = subfolders[i].split(\'/\')[8]\n  index = encode_label(code)\n  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")\n\n  print(code)\n  print(index)\n\n  i = 0\n  for pic in txtfiles:\n    with rasterio.open(pic, "r") as img:\n      arr[i] = np.moveaxis(img.read(), 0, 2)\n      i += 1\n      \n  data[index] = arr'

In [ ]:
#Save the files as .npy

'''for i in range(len(data)):
  keys_list = list(data)
  key = keys_list[i]
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)'''

"for i in range(len(data)):\n  keys_list = list(data)\n  key = keys_list[i]\n  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)"

In [8]:
#Reload the .npy files

import os
#path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
path = '/Volumes/GoogleDrive/My Drive/ML/data/dataset/trainset'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in tqdm(range(len(npyfiles))):
  path = npyfiles[i]
  arr = np.load(path)
  index = path.split('/')[8].split('_')[1][5]
  index = int(index)
  data[index] = arr

In [11]:
from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        id = img_id - 1
        img = data[label_encode][id]
        img_rgb = img[:, :, [4,3,2]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [12]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),              
    transforms.ConvertImageDtype(torch.float),
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [13]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float),torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_ds = EuroSAT(data_df_train, BASE_PATH, train_transform)
print(len(train_ds))

27000


In [15]:
def normalize_for_display(band_data):
    """Normalize multi-spectral imagery across bands.
    The input is expected to be in HxWxC format, e.g. 64x64x13.
    To account for outliers (e.g. extremly high values due to
    reflective surfaces), we normalize with the 2- and 98-percentiles
    instead of minimum and maximum of each band.
    """
    band_data = np.array(band_data)
    lower_perc = np.percentile(band_data, 2, axis=(0,1))
    upper_perc = np.percentile(band_data, 98, axis=(0,1))
    
    return (band_data - lower_perc) / (upper_perc - lower_perc)

In [48]:
img = data[5][1]
normalized_img = normalize_for_display(img)
fig, axs = plt.subplots(4,4, figsize=(15,15))

band_idx = 0
bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12", "B13"]

for i in range(4):
    for j in range(4):
        if band_idx < 13:
            axs[i,j].imshow(normalized_img[:, :, band_idx], cmap="gray")
            axs[i,j].set_title(bands[band_idx])
        else:
            axs[i,j].imshow(np.ones((64,64,3)), cmap="gray")
        
        axs[i,j].axis(False)
        band_idx += 1
        
plt.tight_layout()
plt.show()

NameError: ignored

In [ ]:
rgb_img = normalized_img[:, :, [2]]
fig, ax = plt.subplots(1, figsize=(5,5))
ax.imshow(rgb_img)
ax.set_title(f"{label}")
ax.axis(False)
plt.tight_layout()

plt.show()

In [14]:
train_size = int(0.8 * len(data_df_train))
test_size = len(data_df_train) - train_size
train_ds, validation_ds = torch.utils.data.random_split(train_ds, [train_size, test_size], generator=torch.Generator().manual_seed(seed_value))

In [37]:
print(len(train_ds))
print(len(validation_ds))

21600
5400


Model

In [16]:
pretrainedBool = True
model = models.resnet50(pretrained=True)
if pretrainedBool:
    for param in model.parameters():
        param.requires_grad = False
n_inputs = model.fc.in_features
model.fc = nn.Sequential(
                      nn.Linear(n_inputs, 256),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(256, 10),
                      nn.LogSoftmax(dim=1)
                        )

In [11]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

NameError: ignored

In [19]:
model = model.to(device)

In [40]:
# define the optimization criterion / loss function
ce_loss = nn.CrossEntropyLoss()

In [41]:
ce_loss = ce_loss.to(device)

In [57]:
config={
"epochs": 1,
"mini_batch_size": 64,
"learning_rate": 1e-4,
"opt_func": torch.optim.Adam,
"milestones": [50,75,90],
"weight_decay": 0,
}

In [56]:
train_dl = DataLoader(train_ds, batch_size=config.get("mini_batch_size"), shuffle=True)

In [58]:
run = wandb.init(project="ml-challenge", entity="nicikess", config=config)

In [63]:
epochs = config.get("epochs")
learning_rate = config.get("learning_rate")
weight_decay = config.get("weight_decay")
opt_func = config.get("opt_func")
milestones = config.get("milestones")
max_learning_rate = config.get("max_learning_rate")

train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay)

loss,▄▄▂▄▄█▄▄▃▃▆▂▁▂▅▆▅▅▇▁▄▃▃▃▄▃▂▄▇▃▄▆▄▃▂▂▅▁▃▄
loss,0.29436


[LOG 20220512-15:48:29] epoch: 0 train-loss: 0.4273633391282262


In [73]:
import torch
torch.empty_cache()

AttributeError: ignored

In [61]:
def train(model,train_dl,epochs,learning_rate,opt_func,milestones,weight_decay):
  
  # init collection of training epoch losses
  train_epoch_losses = []

  # set the model in training mode
  model.train()

  optimizer = opt_func(model.fc.parameters(),lr=learning_rate, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=None, gamma=0.1)

  # train the CIFAR10 model
  for epoch in range(epochs):
      
      # init collection of mini-batch losses
      train_mini_batch_losses = []

      # iterate over all-mini batches
      for i, (images, labels) in enumerate(train_dl):

          #print(images)

          # push mini-batch data to computation device
          images = images.to(device)
          labels = labels.to(device)

          # run forward pass through the network
          output = model(images)
          
          # determine classification loss
          loss = ce_loss(output, labels)

          wandb.log({"loss": loss})
          
          # run backward pass
          loss.backward()
          
          # update network paramaters
          optimizer.step()

          # reset graph gradients
          optimizer.zero_grad()
          
          # collect mini-batch reconstruction loss
          train_mini_batch_losses.append(loss.data.item())

      # determine mean min-batch loss of epoch
      train_epoch_loss = np.mean(train_mini_batch_losses)

      #Added
      #result = evaluate(model, vali_dataloader)
      #scheduler.step(result['val_loss'])
      
      # print epoch loss
      now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
      print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
      
      #if (epoch%10 == 0): 
          # set filename of actual model
      model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))
          # save current model to GDrive models directory
      torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))

      # determine mean min-batch loss of epoch
      train_epoch_losses.append(train_epoch_loss)

In [21]:
resnet50_vali_dataloader = DataLoader(validation_ds, batch_size=5400)

In [21]:
# Display image and label.
validation_features, validation_labels = next(iter(resnet50_vali_dataloader))
print(f"Labels batch shape: {validation_labels.size()}")

Labels batch shape: torch.Size([5400])


In [22]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_0.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [ ]:
# set model in evaluation mode
best_model.eval()

In [ ]:
#Validation
#next() returns a tupel: tensor,label
predictions = torch.argmax(best_model(iter(resnet50_vali_dataloader).next()[0]), dim=1)

In [1]:
list_tens = predictions.tolist()

for i in range(len(list_tens)):
  list_tens[i] = decode_target(list_tens[i])

NameError: ignored

In [ ]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

In [ ]:
from sklearn import metrics
acc = metrics.accuracy_score(validation_labels.detach().cpu(), predictions.detach().cpu())

In [ ]:
wandb.log({"acc": acc})

In [ ]:
#Finish run
run.finish()

In [ ]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/testset/'
npyfiles = []
datatest = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in range(len(npyfiles)):
  path = npyfiles[i]
  arr = np.load(path)
  index = int(path.split('_')[1].split('.')[0])
  datatest[index] = arr

In [ ]:
from pathlib import Path
class EuroSATTest(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        img = datatest[idx]
        img = img[:, :, [4,3,2]]
        img = np.float32(img)
        if self.transform:
            img = self.transform(img)
        return img

In [ ]:
#Test data
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_ds = EuroSATTest(data_df_test, BASE_PATH, transf)
print(len(test_ds))

In [ ]:
img = datatest[3]
normalized_img = normalize_for_display(img)
fig, axs = plt.subplots(4,4, figsize=(15,15))

band_idx = 0
bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B10","B11","B12", "B13"]

for i in range(4):
    for j in range(4):
        if band_idx < 13:
            axs[i,j].imshow(normalized_img[:, :, band_idx], cmap="gray")
            axs[i,j].set_title(bands[band_idx])
        else:
            axs[i,j].imshow(np.ones((64,64,3)), cmap="gray")
        
        axs[i,j].axis(False)
        band_idx += 1
        
plt.tight_layout()
plt.show()

In [ ]:
resnet50_test_dataloader = DataLoader(test_ds, batch_size=10_000, num_workers=2, pin_memory=True)

In [ ]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_4.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

In [ ]:
# set model in evaluation mode
best_model.eval()

In [28]:
#Test
predictions = torch.argmax(best_model(iter(resnet50_test_dataloader).next().to(device)), dim=1)

NameError: ignored

In [ ]:
list_tens = predictions.tolist()

for i in range(len(list_tens)):
  list_tens[i] = decode_target(list_tens[i])

In [ ]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['HerbaceousVegetation', 'Industrial', 'Forest', 'SeaLake', 'Residential', 'Highway', 'PermanentCrop', 'River', 'Pasture', 'AnnualCrop'])
dict_values([632, 497, 627, 600, 580, 576, 412, 468, 392, 616])


In [ ]:
from sklearn import metrics
metrics.accuracy_score(validation_labels.detach().cpu(), predictions.detach().cpu())

0.8346296296296296

In [ ]:
test_indices_path = '/content/drive/MyDrive/ML/data/dataset/test.csv'

final_submission_file = pd.read_csv(test_indices_path)
prediction_list = []

# turning numeric predictions to strings (as in kaggle example submission)
for p in predictions:
  prediction_list.append(decode_target(p.item()))

final_submission_file["label"] = prediction_list
### source with alternatives methods for appending to csv should this fail: https://stackoverflow.com/questions/26666919/add-column-in-dataframe-from-list
final_submission_file.to_csv('/content/drive/MyDrive/ML/data/dataset/final_submission.csv', index=None)